I'm sharing a summary of the code I'm working. I'm showing only a "P" class in this code, but am using others in my code. Thank you in advance for your cooperation in research and all support.

# Spatial Relations classification

In this study which makes the assumption that each example is assigned to one and only one label.

### Logical Consequence in LTN

The essence of reasoning is to determine if a closed formula $\phi$ is the logical consequence of a knowledgebase $(\mathcal{K},\mathcal{G}_\theta,\Theta)$, where $\mathcal{K}$ denotes the set of rules in the knowledgebase and $\mathcal{G}_\theta$ denotes a grounding that depends on some parameters $\theta \in \Theta$.

The notion of logical consequence is adapted to Real Logic as follows:
- In classical logic (boolean truth values), a formula $\phi$ is the logical consequence of a knowledgebase $\mathcal{K}$ if, for every interpretation (or model) that verifies the formulas in $\mathcal{K}$, $\phi$ is verified;
- In Real Logic (fuzzy truth values), a formula $\phi$ is the logical consequence of $(\mathcal{K},\mathcal{G}_\theta,\Theta)$ if, for every grounding $\mathcal{G}_\theta$ such that $\mathrm{SatAgg}_{\phi'\in\mathcal{K}}\mathcal{G}_{\theta}(\phi') \geq q $, we have $\mathcal{G}_\theta(\phi)\geq q$, where $q$ is a fixed satisfaction threshold. 

**Reasoning by refutation**:   one seeks to find out a counterexample of a grounding that does satisfy the knowledgebase $\mathcal{K}$ but not the formula $\phi$ (given the threshold $q$). A directed search for such examples is performed using a different learning objective.

Imports

In [1]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logictensornetworks as ltn

# Data

Load the example. Each bb (bounding box) identify in the image by YOLO will be a constant in LTN.
<x; y; h; w; relation >

In [2]:
df_train = pd.read_csv("data2.csv")
df_test = pd.read_csv("data2.csv")
print(df_train.head(5))

    x1   y1   w1   h1    x2  y2   w2    h2 relation
0  525  793  500  386  1104  58  496  1085        D
1  548  803  493  391  1140  57  471  1090        D
2  546  817  505  392  1137  46  475  1109        D
3  575  820  484  391  1144  65  463  1088        D
4  568  818  504  397  1152  41  457  1126        D


In [3]:
labels_train = pd.read_csv("data3.csv")
labels_test = pd.read_csv("data3.csv")
print(labels_train.head(5))

    x1   y1   w1   h1    x2  y2   w2    h2
0  525  793  500  386  1104  58  496  1085
1  548  803  493  391  1140  57  471  1090
2  546  817  505  392  1137  46  475  1109
3  575  820  484  391  1144  65  463  1088
4  568  818  504  397  1152  41  457  1126


Batch size is a term used in machine learning and refers to the number of training examples utilized in one iteration.

similar like multiclass

In [4]:
features = df_train[['x1','y1','w1','h1','x2','y2','w2','h2']]
labels_position = df_train['relation']

batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((features,labels_position)).batch(batch_size)
#ds_test = tf.data.Dataset.from_tensor_slices((features[160:],labels_sex[160:],labels_color[160:])).batch(batch_size)

# LTN

Predicate with softmax `P(x,class)` defined by "ltn.Predicate"

Constants to index/iterate on the classes

In [5]:
#position classification
class_P = ltn.constant(0, True)

Operators

In [6]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=5),semantics="exists")
and_luk = ltn.fuzzy_ops.And_Luk()

We define the membership predicate $ModelSpatialRelation(x,l)$, where $x$ is an individual and $l$ is a onehot label to denote the three classes. $ModelSpatialRelation$ is approximated by a simple MLP. The last layer, that computes probabilities per class, uses a `softmax` activation, ensuring that the classes are mutually-exclusive.

To evaluate the grounding of each formula, one has to define the grounding of the non-logical symbols and of the operators.

In [7]:
class ModelSpatialRelation(tf.keras.Model):
    def __init__(self):
        super(ModelSpatialRelation, self).__init__()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu) #P
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.elu) #PO
        self.dense3 = tf.keras.layers.Dense(5, activation=tf.nn.elu) #D
        self.dense4 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)
        
    def call(self, inputs, training=False):
        """bb1,bb2 | label: onehot label"""
        bb1, bb2 = inputs[0], inputs[1] # multiple arguments are passed as a list
        label = predicate_position(bb1,bb2)
        bb12 = tf.concat([bb1,bb2],axis=1) # axis=0 is the batch axis
        bb12 = self.dense1(bb12) #layer P
        bb12 = self.dense2(bb12) #layer PO
        bb12 = self.dense3(bb12) #layer D
        prob = self.dense4(bb12) #layer unitting all
        return tf.math.reduce_sum(prob*label,axis=1)

#logits_model = ModelSpatialRelation(3)
#p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(logits_model,single_label=True))
SpatialRelation = ltn.Predicate(ModelSpatialRelation())
Sim_SR = ltn.Predicate.Lambda(lambda args: tf.exp(bb1,bb2))

W1110 11:20:33.050862 10176 deprecation.py:506] From C:\Users\Milena\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**It is important to always run (forward pass) the knowledgebase once before training, as Tensorflow initializes weights and compiles the graph during the first call.**

Axiomas

In [8]:
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=2)

@tf.function
def axioms(features, labels_position, training=False):
    x = ltn.variable("x",features)
    
    #possible fix
    mask = (labels_position=="P")
    print(type(mask))
    tensor_P = features[mask]
    x_P = ltn.variable("P", tensor_P)  

    axioms = [
        Forall(x_P,p([x_P,class_P],training=training)),
        #insert my rules here x -> y 
    ]
    axioms = tf.stack(axioms)
    sat_level = formula_aggregator(axioms)
    return sat_level, axioms

Initialize all layers and the static graph

In [9]:
for _data, _labels in ds_train:
    print(axioms(_data, _labels)[0])
    break

<class 'bool'>


ValueError: in converted code:

    <ipython-input-8-e38966c37e78>:10 axioms *
        tensor_P = features[mask]
    C:\Users\Milena\Anaconda3\lib\site-packages\tensorflow\python\ops\array_ops.py:600 _slice_helper
        return boolean_mask(tensor=tensor, mask=slice_spec)
    C:\Users\Milena\Anaconda3\lib\site-packages\tensorflow\python\ops\array_ops.py:1365 boolean_mask
        raise ValueError("mask cannot be scalar.")

    ValueError: mask cannot be scalar.


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.